# RLC - Reptile pretraining
Notebook to run experiments on the RLC circuit dataset.

Before running this notebook, you need to run the pretraining run. To do so, execute the following command from the repository root directory:
`python tsfewshot/run_scheduler.py train --directory experiments/nl-rlc/configs/ --runs-per-gpu 4 --gpu-ids 0` or `python tsfewshot/run.py train --config-file experiments/nl-rlc/configs/reptile-config.yml --gpu 0`

The first will run all config files in the specified directory, with 4 runs in parallel on GPU 0, the latter only the Reptile pretraining run.

In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import sys
from collections import defaultdict
from pathlib import Path

from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', datefmt='%I:%M:%S')

import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
import pandas as pd
import scipy
import torch
import os
from tqdm import tqdm

sys.path.append('../..')
from tsfewshot.config import Config
from tsfewshot import plot, analyses

np.random.seed(0)
torch.random.manual_seed(0)

DEVICE = 'cuda:0'
SUPPORT_SIZES = [10, 20, 30, 50, 70, 100]
QUERY_SIZE = 2000

CODE_DIR = Path('../..')
DATA_DIR = 'experiments/nl-rlc/data/rlc_dataset-resistor_range1_14-capacitor_range100_800-inductor_range20_140'

In [2]:
# get the run directories from the meta-training phase
RUN = Path('runs/reptile/rlc-ode-3trtraj-reptile-50support10batch5steplr0.001innerlr0.001-finetune5-seed0').resolve()

In [ ]:
# create tasks on which we will generate the finetuning trajectories that
# will be used to calculate the preconditioning matrix
finetune_tasks = sorted(f'val/{d.name}' for d in (Path(DATA_DIR) / 'val').glob('*.npy'))
len(finetune_tasks)

In [ ]:
# create finetuning configurations for each of the above finetune_tasks
n_ft_tasks = len(finetune_tasks)
finetune_lrs = [5e-6]
# generate finetune configs
for run in [RUN]:
    best_epoch_file = run / 'best_epoch.txt'
    if not best_epoch_file.exists():
        print(f'best-epoch file {best_epoch_file} not found.')
        continue
    with best_epoch_file.open('r') as fp:
        epoch = int(fp.read())
    cfg = Config(run / 'config.yml')
    for flr in finetune_lrs:
        finetune_cfg = f"""
experiment_name: {run.name}-adam_flr{flr}-n_fts{n_ft_tasks}-NNNN
val_datasets: 
 - VVVV#0#3
test_datasets: 
 - VVVV#3#5
finetune_lr: {flr}
optimizer: adam
finetune_epochs: 30
early_stopping_patience: 5
eval_every: 1
save_every: -1
checkpoint_path: {str(run.absolute())}/model_epoch{str(epoch).zfill(3)}.p
base_run_dir: {str(run.absolute())}
run_dir: {str(run.absolute())}/finetune_adam_flr{flr}_n_fts{n_ft_tasks}
finetune: false
training_setup: supervised
predict_last_n: 256
seq_length: 256
val_n_random_datasets: null
batch_size: 16
"""

        (run / f'finetune_adam_flr{flr}_n_fts{n_ft_tasks}/configs').mkdir(exist_ok=True, parents=True)
        print(run / f'finetune_adam_flr{flr}_n_fts{n_ft_tasks}/configs')
        for task in finetune_tasks:
            task_finetune_cfg = finetune_cfg.replace('VVVV', task).replace('NNNN', task.replace('/', ''))
            with (run / f'finetune_adam_flr{flr}_n_fts{n_ft_tasks}/configs/{task.replace("#", "").replace("/", "")}.yml').open('w') as f:
                f.write(task_finetune_cfg)

Next, you need to start the created finetuning runs. For each pretraining run, execute:
`python tsfewshot/run_scheduler.py finetune --directory /path/to/run/finetune_adam_flr5e-6_n_fts512/configs/ --gpu-ids 0 --runs-per-gpu 5`
(adapt gpu-ids and runs-per-gpu according to the number and size of your GPU(s)).

After all runs are completed, calculate the preconditioning matrix for each support size:
`python pca.py --base-dir /path/to/run --finetune-dirs /path/to/run/finetune_adam_flr5e-6_n_fts512/rlc* --epoch -1`.
This will pickle the preconditioning matrix to `/path/to/run/pca/`.

In [ ]:
ft_epochs = defaultdict(lambda: 500)

eval_every = list(range(20)) + list(range(20, 501, 20))
inner_seeds = [0]

type_specs = [
    'normal',  # SGD finetuning
    'pca',  # our method
]

gridsearch_dir = 'optimizeFinetune'

def combinations():
    combinations = []
    for type_spec in type_specs:
        if 'pca' in type_spec:
            lrs = [1e-2, 1e-3, 1e-4, 1e-5, 1e-6]  
        elif 'normal' in type_spec:
            lrs = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
        elif 'jfr' in type_spec:
            lrs = [1.0] # dummy, this adaption method does not use gradient descent, hence no lr
        for lr in lrs:
            for support in SUPPORT_SIZES:
                combinations.append({'support': support,
                                     'lr': lr,
                                     'type': type_spec,
                                     'noise': 0.0,
                                     'sparsity': None,
                                     'pca_interpolate': 1.0,
                                    })
    return combinations
    
len(combinations())

In [ ]:
# create gridsearch for optimal learning rate and number of update steps during finetuning.
new_runs = analyses.create_gridsearch_dirs(RUN,
                                            combinations(),
                                            gridsearch_dir,
                                            inner_seeds,
                                            n_trajectories=-1,
                                            query_size=QUERY_SIZE,
                                            ft_epochs=ft_epochs,
                                            eval_every=eval_every,
                                            save_predictions=True,
                                            optimizer='sgd-squared',
                                            val_datasets=finetune_tasks[::2], # gridsearch on half of the finetune_tasks
                                            init_epoch=None, # uses best epoch
                                            pca_file_path=RUN / 'pca/pca-torchcenterFalseNonecomponentspathNoneimprovedFalseusestepsNonelayerwiseFalse.p')

[r.name for r in new_runs]

Next, run the created gridsearch configurations via:
`python tsfewshot/run_scheduler.py eval --directory /path/to/run/optimizeFinetune/ --epoch -1 --split val --gpu-ids 0 --runs-per-gpu 5`.

In [5]:
best_ft_options = {}
best_ft_epochs = {}
gridsearch_results = {}

In [ ]:
# load the gridsearch results and calculate the best learning rate and number of update steps
# for each support size and each method.
# For the best configuration of each support size and method, we create a final finetuning experiment
# that we'll use to analyze the results.
test_tasks = sorted(f'test/{d.name}' for d in (Path(DATA_DIR) / 'test').glob('*.npy'))
new_run_dirs, best_ft_options, best_ft_epochs, gridsearch_results = \
    analyses.create_final_finetune_dirs([RUN],
                                          SUPPORT_SIZES,
                                          combinations(),
                                          inner_seeds,
                                          gridsearch_dir,
                                          test_tasks,
                                          best_ft_options=best_ft_options,
                                          best_ft_epochs=best_ft_epochs,
                                          n_results=256,
                                          metric_aggregation='median',
                                          metric_name='mse')
[r.name for r in new_run_dirs]

In [ ]:
_ = plot.plot_gridsearch(gridsearch_results)
_ = plt.ylim(0.0, 0.2)

Next, run the final finetuning experiments:
`python tsfewshot/run_scheduler.py eval --directory /path/to/run/optimizeFinetune_finalFinetune/ --epoch -1 --split test --gpu-ids 0 --runs-per-gpu 5`.

In [ ]:
# load the result from final finetuning
metrics = None
metrics = analyses.get_final_metrics([RUN],
                                     noises=[0.0],
                                     support_sizes=SUPPORT_SIZES,
                                     combinations=combinations(),
                                     best_ft_options=best_ft_options,
                                     inner_seeds=inner_seeds,
                                     query_size=QUERY_SIZE,
                                     n_trajectories=-1,
                                     test_tasks=test_tasks,
                                     gridsearch_dir=gridsearch_dir,
                                     init_epoch=None,
                                     metrics=metrics,
                                     metric_name='mse',
                                     metric_file_name='mse_rmse',
                                     metric_aggregation='median',
                                     n_dataset_evals = None,
                                     no_ft_eval=True)

In [9]:
# dump results so we don't have to recalculate each time we run this notebook
pickle.dump(metrics, (RUN / 'metrics.p').open('wb'))
pickle.dump(best_ft_options, (RUN / 'best_ft_options.p').open('wb'))
pickle.dump(best_ft_epochs, (RUN / 'best_ft_epochs.p').open('wb'))

In [4]:
# load metrics when re-executing the notebook, so we don't have to recalculate
metrics = pickle.load((RUN / 'metrics_.p').open('rb'))
best_ft_options = pickle.load((RUN / 'best_ft_options.p').open('rb'))
best_ft_epochs = pickle.load((RUN / 'best_ft_epochs.p').open('rb'))

In [ ]:
df_norms = pd.DataFrame({k: v['mse'] for k, v in metrics[0.0].items()}, dtype=float)
df_norms.columns.names = ['support', 'type', 'seed']

# results table
display_df = df_norms.copy()
median = display_df.median(axis=0)
mean = display_df.mean(axis=0)
display_df.loc[' rank'] = display_df.groupby('support', axis=1).rank(axis=1).median(axis=0)
display_df.loc[' median'] = median
display_df.loc[' mean'] = mean
display(display_df.sort_index().style.background_gradient('Greens_r', axis=1).highlight_null('white'))

In [6]:
from matplotlib import rc
rc('text', usetex=True)

In [ ]:
# plot results
figsize = (7,4)
f = plot.plot_support_vs_mse(df_norms,
                             SUPPORT_SIZES,
                             ranks=False,
                             figsize=figsize,
                             aggregation='median',
                             title='rlc', 
                             exclude_types=[],
                             metric_name='MSE')
plt.ylim(0,0.1)
f2 = plot.plot_support_vs_mse(df_norms,
                              SUPPORT_SIZES,
                              ranks=True,
                              figsize=figsize,
                              title='rlc')

In [7]:
# significance test
from scipy.stats import wilcoxon
p = {}
noise = 0.0
sig_df = df_norms.groupby(['support', 'type'], axis=1).agg(lambda s: s.mean(skipna=False))
best_typs = {support: sig_df.loc[:, support].median().idxmin(axis=1) for support in SUPPORT_SIZES}
for support, typ in sig_df.columns:
    if typ == f'{best_typs[support]} noise 0.0 ':
        p[(support, typ)] = np.nan
        continue
    if ((sig_df.loc[:, (support, typ)] - sig_df.loc[:, (support, best_typs[support])]) == 0).all():
        p[(support, typ)] = np.nan
        continue
    p[(support, typ)] = wilcoxon(sig_df.loc[:, (support, typ)], sig_df.loc[:, (support, best_typs[support])])[1]
sig_df = pd.DataFrame(p, index=['p-value']).T
sig_df.index.names = ['support', 'type']
display(sig_df.reset_index().pivot(index='type', columns='support', values='p-value').style.format('{:.3e}'))

support,10,20,30,50,70,100
type,,,,,,
no-finetune,4.947e-07,1.425e-11,1.491e-21,3.469e-28,9.894e-31,1.239e-30
normal noise 0.0,1.520e-07,8.573e-13,6.067e-21,2.597e-23,4.771e-23,3.926e-24
pca noise 0.0,nan,nan,nan,nan,nan,nan


In [ ]:
# Plot eigenvalue distribution
pca = pickle.load((RUN / 'pca/pca-torchcenterFalseNonecomponentspathNoneimprovedFalseusestepsNonelayerwiseFalse.p').open('rb'))
eigenvalues = pca['s']**2 / (pca['u'].shape[0] - 1)
plt.plot(eigenvalues)
plt.ylabel('Eigenvalue')
plt.xlabel('Component')
plt.grid(alpha=0.6)
plt.yscale('log')
plt.tight_layout()

In [ ]:
run = RUN
finetune_dirs = list((run / f'finetune_adam_epoch{init_epoch}').glob('rlc*/'))  # TODO init_epoch
cfg = Config(run / 'config.yml')
plt.figure(figsize=(14,7))
f, ax = plot.plot_deltas_rank(cfg, finetune_dirs,
                              tols=[0],
                              ax=plt.gca(),
                              random_baseline=True,
                              n_repeats=1,
                              task_steps=1,
                              within_task_steps=None,
                              epoch_steps=None,
                              use_erank=True,
                              init_epoch=-1,
                              plot_val_metric=False)
plt.xscale('log')
plt.tight_layout()